# Basic occupancy analysis

COMING SOON

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import hillmaker as hm

## A prototypical example - occupancy analysis of a hospital Short Stay Unit

Patients flow through a short stay unit (SSU) for a variety of procedures, tests or therapies. Let's assume patients can be classified into one of five categories of patient types: ART (arterialgram), CAT (post cardiac-cath), MYE (myelogram), IVT (IV therapy), and OTH (other). We are interested in occupancy statistics by time of day and day of week to support things like staff scheduling and capacity planning.

From one of our hospital information systems we were able to get raw data about the entry and exit times of each patient and exported the data to a csv file. We call each row of such data a *stop* (as in, the patient stopped here for a while). Let's take a peek at the data by first reading the csv file into a pandas `DataFrame`.

In [3]:
ssu_stopdata = './data/ssu_2024.csv'
stops_df = pd.read_csv(ssu_stopdata, parse_dates=['InRoomTS','OutRoomTS'])
stops_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59877 entries, 0 to 59876
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   PatID      59877 non-null  int64         
 1   InRoomTS   59877 non-null  datetime64[ns]
 2   OutRoomTS  59877 non-null  datetime64[ns]
 3   PatType    59877 non-null  object        
 4   LOS_hours  59877 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [30]:
ssu_all = hm.Scenario(scenario_name='ssu_all',
                      data=stops_df,
                      in_field='InRoomTS',
                      out_field='OutRoomTS',
                      start_analysis_dt='2024-01-02',
                      end_analysis_dt='2024-09-30',
                      cat_field='PatType',
                      bin_size_minutes=60,
                      verbosity=1,
                     )

In [31]:
print(ssu_all)

Required inputs
-------------------------
scenario_name = ssu_all
data =
       PatID            InRoomTS           OutRoomTS PatType  LOS_hours
0          1 2024-01-01 07:44:00 2024-01-01 09:20:00     IVT   1.600000
1          2 2024-01-01 08:28:00 2024-01-01 11:13:00     IVT   2.750000
2          3 2024-01-01 11:44:00 2024-01-01 12:48:00     MYE   1.066667
3          4 2024-01-01 11:51:00 2024-01-01 21:10:00     CAT   9.316667
4          5 2024-01-01 12:10:00 2024-01-01 12:57:00     IVT   0.783333
...      ...                 ...                 ...     ...        ...
59872  59873 2024-09-30 19:31:00 2024-09-30 20:34:00     IVT   1.050000
59873  59874 2024-09-30 20:23:00 2024-09-30 22:22:00     IVT   1.983333
59874  59875 2024-09-30 21:00:00 2024-09-30 23:22:00     CAT   2.366667
59875  59876 2024-09-30 21:57:00 2024-10-01 01:58:00     IVT   4.016667
59876  59877 2024-09-30 22:45:00 2024-10-01 03:18:00     CAT   4.550000

[59877 rows x 5 columns]
in_field = InRoomTS
out_field = OutRo

In [ ]:
ssu_all.make_hills()

In [ ]:
ssu_all.hills.keys()